In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from apps.genetic_algorithms.genetic_program import FitnessFunction
from apps.genetic_algorithms.gp_artemis import ExperimentManager
from IPython.core.display import HTML, Javascript

CONDENSED_COLS = ['individual_str', 'transaction_currency', 'counter_currency', 'profit_percent', 'benchmark_profit_percent']

%matplotlib inline
pd.set_option('display.max_columns', 500)

from apps.doge.doge_train_test import DogeTrainer
import time
end_timestamp = 1548806400 #int(time.time())  # UTC timestamp
start_timestamp = end_timestamp - 60*60*24
ticker = 'BTC_USDT'

e = DogeTrainer.run_training(start_timestamp, end_timestamp, ticker)
            


### Looking at performance per training dataset and across all experiment variants
For each training dataset in the training collection, we get a performance dataframe sorted by profit. The dataframes and best individuals in them are shown below.

In [ ]:
performance_dfs = e.get_joined_performance_dfs_over_all_variants()

In [ ]:
len(e.training_data[0].indicators['rsi'])
#e.training_data[0].start_time
e.training_data[0].end_time

We can explore each of the performance dataframes in more detail.

In [ ]:
pd.set_option('display.max_colwidth', -1)
performance_dfs[0][CONDENSED_COLS]

In [ ]:
e.performance_df_row_info(performance_dfs[0].iloc[0])

### Best performing individuals across all variants and datasets
Each experiment variant will have a winner, the doge baby that had the best fitness value. We can get these winners by invoking the command below:

In [ ]:
best_individuals_across_variants_and_datasets = e.get_best_performing_across_variants_and_datasets(e.training_data, sort_by=["fitness_value"], top_n_per_variant=20)

In [ ]:
pd.set_option('display.max_colwidth', -1)
best_individuals_across_variants_and_datasets.drop(['individual','variant','evaluations'], axis=1)




### How did this individual perform on training sets?

In [ ]:
best_overall = best_individuals_across_variants_and_datasets.iloc[0]
evaluations = best_overall.evaluations
for evaluation in evaluations:
    e._print_individual_info(best_overall.individual, evaluation)

### Let's see its performance on the validation set...

In [ ]:
# Build the validation set
from apps.backtesting.data_sources import Data, DB_INTERFACE
validation_start = DB_INTERFACE.get_nearest_db_timestamp(end_timestamp, 'BTC_USDT')
validation_end = DB_INTERFACE.get_nearest_db_timestamp(end_timestamp+60*60*24, 'BTC_USDT')
from apps.backtesting.data_sources import Data, DB_INTERFACE
validation = Data(
    start_time=validation_start, 
    end_time=validation_end, 
    ticker='BTC_USDT', 
    horizon=e.training_data[0].horizon, 
    start_cash=1000, 
    start_crypto=0, 
    exchange='binance', 
    database=DB_INTERFACE
)

evaluation = e._build_evaluation_object(best_overall.individual, best_overall.variant, validation)
e._print_individual_info(best_overall.individual, evaluation)

In [ ]:
perf = e.evaluate_individual_on_data_collection(best_overall.individual, best_overall.variant, e.validation_data)
#perf = perf.drop(['individual'], axis=1)
perf[CONDENSED_COLS]

In [ ]:
doge = DogeDNACanvas(best_overall.individual,"container")
HTML(doge.create_container())

In [ ]:
Javascript(doge.show())